In [1]:
import os
from google.colab import drive
 
drive.mount('/content/gdrive')
%cd '/content/gdrive/My Drive/tinkerhub'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/tinkerhub


In [2]:
!pip install facenet_pytorch

In [3]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from PIL import Image

In [4]:

mtcnn = MTCNN(image_size=240, margin=0, min_face_size=20) # initializing mtcnn for face detection
resnet = InceptionResnetV1(pretrained='vggface2').eval() # initializing resnet for face img to embeding conversion

dataset=datasets.ImageFolder('train') # photos folder path 
idx_to_class = {i:c for c,i in dataset.class_to_idx.items()} # accessing names of peoples from folder names

def collate_fn(x):
    return x[0]

loader = DataLoader(dataset, collate_fn=collate_fn)

face_list = [] # list of cropped faces from photos folder
name_list = [] # list of names corrospoing to cropped photos
embedding_list = [] # list of embeding matrix after conversion from cropped faces to embedding matrix using resnet

for img, idx in loader:
    face, prob = mtcnn(img, return_prob=True) 
    if face is not None and prob>0.90: # if face detected and porbability > 90%
        emb = resnet(face.unsqueeze(0)) # passing cropped face into resnet model to get embedding matrix
        embedding_list.append(emb.detach()) # resulten embedding matrix is stored in a list
        name_list.append(idx_to_class[idx]) # names are stored in a list
        


In [5]:
data = [embedding_list, name_list]
torch.save(data, 'data2.pt') # saving data.pt file

In [6]:
mtcnn = MTCNN(image_size=240, margin=0, min_face_size=20) # initializing mtcnn for face detection
resnet = InceptionResnetV1(pretrained='vggface2').eval() # initializing resnet for face img to embeding conversion

In [7]:
torch.save(mtcnn,"data1.pt")
torch.save(resnet,"data3.pt")

### Matching face id of the given photo with available data from data2.pt file

In [28]:

def face_match(img_path, data_path): # img_path= location of photo, data_path= location of data.pt 
    # getting embedding matrix of the given img
    img = Image.open(img_path)
    face, prob = mtcnn(img, return_prob=True) # returns cropped face and probability
    emb = resnet(face.unsqueeze(0)).detach() # detech is to make required gradient false
    saved_data = torch.load('data2.pt') # loading data.pt file
    embedding_list = saved_data[0] # getting embedding data
    name_list = saved_data[1] # getting list of names
    dist_list = [] # list of matched distances, minimum distance is used to identify the person
    
    for idx, emb_db in enumerate(embedding_list):
        dist = torch.dist(emb, emb_db).item()
        dist_list.append(dist)
        
    idx_min = dist_list.index(min(dist_list))
    return (name_list[idx_min], min(dist_list))


result = face_match('download (16).jpg', 'data2.pt')

print('Face matched with: ',result[0], 'With distance: ',result[1])

Face matched with:  unknown With distance:  0.0


In [37]:
y_test,y_pred = [],[]
fmammooka , fmohanlal , funknown= [] , [] ,[]


In [38]:
from os import walk

In [39]:
for (dirpath, dirnames, filenames) in walk('test/mammootty'):
    fmammooka.extend(filenames)
    break

In [40]:
for (dirpath, dirnames, filenames) in walk('test/mohanlal'):
    fmohanlal.extend(filenames)
    break

In [41]:
for (dirpath, dirnames, filenames) in walk('test/unknown'):
    funknown.extend(filenames)
    break

In [42]:
for i in fmammooka:
  y_test.append(0)
  result = face_match(f"test/mammootty/{i}", 'data2.pt')
  if(result[0] == "mammootty"):
    y_pred.append(0)
  elif(result[0] == "mohanlal"):
    y_pred.append(1)
  elif(result[0] == "unknown"):
    y_pred.append(2)


In [43]:
for i in fmohanlal:
  y_test.append(1)
  result = face_match(f"test/mohanlal/{i}", 'data2.pt')
  if(result[0] == "mammootty"):
    y_pred.append(0)
  elif(result[0] == "mohanlal"):
    y_pred.append(1)
  elif(result[0] == "unknown"):
    y_pred.append(2)

In [44]:
for i in funknown:
  y_test.append(2)
  result = face_match(f"test/unknown/{i}", 'data2.pt')
  if(result[0] == "mammootty"):
    y_pred.append(0)
  elif(result[0] == "mohanlal"):
    y_pred.append(1)
  elif(result[0] == "unknown"):
    y_pred.append(2)

In [47]:
print(accuracy_score(y_test,y_pred))

0.86


In [46]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.82      0.95      0.88        19
           1       0.88      1.00      0.93        21
           2       1.00      0.40      0.57        10

    accuracy                           0.86        50
   macro avg       0.90      0.78      0.79        50
weighted avg       0.88      0.86      0.84        50

